In [2]:
from os import listdir
from os.path import isfile, join
from PIL import Image

import random
import math
import time
import numpy as np
import cv2
import os
import shutil

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import torch.nn as nn
import torch.nn.functional as F

In [3]:
widthSteering = heightSteering = 140

widthRoad = 600
heightRoad = 200

modelsPath = 'Models'
modelName = 'bestModel.torch'
pathToModel = f'{modelsPath}/{modelName}'

framesPath = 'DrivingFramesStarting'
labeledFrames = 'LabledFrames'


#road crop area
x1R = 400
x2R = 600
y1R = 630
y2R = 1280

#road image size
widthRoad  = y2R - y1R
heightRoad = x2R - x1R

#wheel crop area
x1W = 812
x2W = 932
y1W = 913
y2W = 1033

#wheel image size
widthWheel  = y2W - y1W
heightWheel = x2W-x1W


In [4]:
exist = os.path.exists(modelsPath)
if exist:
    print('Models Directoty Exists')
else:
    print('Models Directory Does not Exists')

exist = os.path.exists(framesPath)
if exist:
    print('Starting Frames Directoty Exists')
else:
    print('Starting Frames Directory Does not Exists')


exist = os.path.exists(labeledFrames)
if not exist:
    os.makedirs(labeledFrames)
    print("Labled Frames Directoty Created")
else:
    print('Labled Frames Directory Exists')
    #Uncomment these two lines if you want to delete the frames folder before adding more frames
    #shutil.rmtree(labeledFrames)
    #os.makedirs(labeledFrames)

Models Directoty Exists
Starting Frames Directoty Exists
Labled Frames Directory Exists


In [25]:
startingFrames = [f for f in listdir(framesPath) if isfile(join(framesPath, f))]

In [26]:
len(startingFrames)

200300

In [7]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.fc = nn.Linear(512, 128)
        
        self.branch_a1 = nn.Linear(128, 32)
        self.branch_a2 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc(x))

        a = F.leaky_relu(self.branch_a1(x))
        
        out1 = self.branch_a2(a)
        
        return out1

In [34]:
#load net
#To Load Pretrained Weights:   weights='ResNet18_Weights.DEFAULT'
resnet18 = torchvision.models.resnet18()
resnet18.fc = nn.Identity()
net_add=net()
model = nn.Sequential(resnet18, net_add)
# Set device GPU or CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
# load model to GPU
model = model.to(device)
model.load_state_dict(torch.load(pathToModel))
model.eval()

count = 0
for file in startingFrames:
    noExtension = file[:-4]
    filePath = f'{framesPath}/{file}'


    fileSize = os.path.getsize(filePath)
    if fileSize>100:
        #load image
        wholeFrame = cv2.imread(filePath)
        

        #wheel angle prediction
        crop = wholeFrame[x1W:x2W, y1W:y2W]
        grayImage = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        (thresh, contrastImg) = cv2.threshold(grayImage, 100, 255, cv2.THRESH_BINARY)
        backtorgb = cv2.cvtColor(contrastImg,cv2.COLOR_GRAY2RGB)
        transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((heightWheel,widthWheel)),tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
        wheel=transformImg(np.array(backtorgb))
        wheel = wheel.to(device).unsqueeze(0)
        with torch.no_grad():
            prediction = model(wheel)
        detectedAngle = prediction.data.cpu().numpy()[0][0]
        realAngle = (180*detectedAngle)-90
        

        # crop to road
        crop = wholeFrame[x1R:x2R, y1R:y2R]

        # resize to 180*56
        resized = cv2.resize(crop, [180,56], interpolation = cv2.INTER_AREA)


        cv2.imwrite(f'{labeledFrames}/{noExtension}_roadcrop_alpha_none_filter_none_angleNorm_{detectedAngle}.jpg', resized)

        #array of all angle values to adjust brightness by
        #alphas = [0.2,0.4,0.6,0.8,1.2,1.4]
        
        #for alpha in alphas:

                #brighness = cv2.convertScaleAbs(resized, alpha, alpha)
                #cv2.imwrite(f'{labeledFrames}/{noExtension}_roadcrop_alpha_{alpha}_filter_none_angleNorm_{detectedAngle}.jpg', brighness)

                #for f in range(2,3):
                   # blur = cv2.blur(brighness,(f,f))

                    #cv2.imwrite(f'{labeledFrames}/{noExtension}_roadcrop_alpha_{alpha}_filter_{f}_angleNorm_{detectedAngle}.jpg', blur)
    count = count + 1
    if count%1000==0:
         print(count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [35]:
augmentedFrames = [f for f in listdir(labeledFrames) if isfile(join(labeledFrames, f))]

In [36]:
len(augmentedFrames)

2485431